# Modelagem preditiva para Análise de Crédito

## Introdução

O presente trabalho visa realizar uma modelagem preditiva, baseada nos pilares
da ciência de dados e do aprendizado de máquina. Para a análise, foi utilizada
base de dados
[default of credit card clients](https://archive.ics.uci.edu/dataset/350/default+of+credit+card+clients)
da UCI, a qual contém cerca de 30000 contas de etiquetadas de maneira binária,
os quais indicam se o cliente conseguiu (ou não) pagar a fatura mínima de
crédito dentro dos ultimos 6 meses.

## Objetivos

O objetivo do presente trabalho se concentra na prática da modelagem preditiva,
bem como um melhor entendimento do problema central, *a análise de crédito*.

Segundo o [Sinfab/RS](https://sinbraf.com.br/voce-sabe-o-que-e-credito-2/), crédito é:

    CRÉDITO é uma fonte adicional de recursos que não são seus, mas obtidos de terceiros (bancos, financeiras, cooperativas de crédito e outros), que possibilita a antecipação do consumo para a aquisição de bens ou contratação de serviços. 

Nesse sentido, modelar crédito é medir o quão confiável o está a vida financeira
de uma conta. Se a conta apresentar indícios de que está passando por um período
menos estável, gostariamos de evitar liberar um crédito alto demais.

# Obetenção dos dados

## Obtendo os dados via rede

para fins de praticidade, escrevi uma pequena automação para baixar os dados
nencessários do site da UCI, descompactá-los e deixar em uma pasta a minha
disposição.

In [7]:
import os
import requests
from zipfile import ZipFile

def download_and_unzip(url, local_folder):
    if not os.path.exists(local_folder):
        os.makedirs(local_folder)

    response = requests.get(url)
    zip_file_path = os.path.join(local_folder, 'downloaded_file.zip')
    
    with open(zip_file_path, 'wb') as zip_file:
        zip_file.write(response.content)

    with ZipFile(zip_file_path, 'r') as zip_ref:
        zip_ref.extractall(local_folder)

    os.remove(zip_file_path)

In [9]:
download_and_unzip(
    url='https://archive.ics.uci.edu/static/public/350/default+of+credit+card+clients.zip',
    local_folder='datasets'
)

## Visualizando uma amostra

In [19]:
import pandas as pd
import matplotlib.pyplot as plt

credit_data = pd.read_excel(
    os.path.join('datasets', 'default of credit card clients.xls'))

credit_data.sample(5)

,Unnamed: 0,X1,X2,X3,X4,X5,X6,X7,X8,X9,...,X15,X16,X17,X18,X19,X20,X21,X22,X23,Y
6584,6584,200000,2,2,2,27,-1,-1,2,0,...,823,-658,1266,2250,0,0,0,1924,0,1
26739,26739,50000,1,2,2,23,1,2,2,2,...,8631,8855,8585,0,2000,1000,500,2,500,1
12143,12143,70000,2,2,2,25,4,3,2,0,...,29540,29505,29428,0,0,2032,1069,1066,1002,1
27188,27188,240000,1,3,2,28,0,0,-1,-1,...,10591,1793,1509,3851,1467,10594,1802,1516,1193,0
8500,8500,260000,2,2,2,35,-1,-1,-1,-1,...,188,194,188,188,188,188,194,188,221,0


# Conhecendo o conjunto de dados

Uma vez obtidos os dados, podemos iniciar nossa análise exploratória.